In [ ]:
import pyemu
import conda_scripts.load_datasets as lsd
import os
import pandas as pd
import numpy as np
import conda_scripts
import forward_run
import matplotlib.pyplot as plt
import conda_scripts.plot_help as ph
from conda_scripts import sv_budget
import matplotlib as mpl
import numpy as np
import shutil
import geopandas as gpd
from shutil import ignore_patterns
import helpers

In [ ]:
workspace = lsd.model_info.get_mod('son', True)
template_ws = "pestaws_IPA"

In [ ]:
import matplotlib.pyplot as plt

%matplotlib inline


# postprocess

In [ ]:
main = 'postprocess'
fold =  'pestaws_IPA'
run_name = 'IPA'
out_folder = os.path.join(main, fold)
if not os.path.exists(out_folder):
    os.makedirs(out_folder, exist_ok =True)

# new folder

In [ ]:
new_run = 'cheese'

### description
`starting with existing run (IPA in this case), simply change weights to re-run prior already ran`

1) copy files from base (lager)
2) remove all pest files
3) load pst from base
4) updates weights
5) set control parameters to 
6) write pst files to new folder



# setup new dir if it doesn't exist

In [ ]:
if not os.path.exists(new_run):
    print(f'creating {new_run}')
    # Copy the directory tree from src to dest, ignoring the .git directory
    shutil.copytree(template_ws, new_run, ignore=ignore_patterns('.git', '.git/*', '.idea/*','mult/*','org/*','*.cbc','*.hds','__pycache__/*','SFR_DELIVERY_BY_WBS.out','sv_model_grid_6layers.lst'))
else:
    print(f"not creating {new_run} because it already exists")

In [ ]:
# load pest from base
base = r"C:\GSP\sv\model\pest_inputs\PEST_PP\pestaws_IPA"
pst = pyemu.Pst(os.path.join(base, f'{run_name}.pst'), resfile = os.path.join(base,f'{run_name}.0.base.rei'))
pst.phi

# get phi contributions from existing prior. then change weights (setting to zero some obs for now). then re-balance weights to this weighting scheme

In [ ]:
og_weights = pst.phi_components

In [ ]:
og_weights['yearly_min_flow_aguacal'] = og_weights['yearly_min_flow_aguacal']*2
og_weights['aguacal_log'] = og_weights['aguacal_log']*2

In [ ]:
vcount = pd.Series(og_weights).to_frame('phi').reset_index().rename(columns = {'index':'obgnme'})


vcount.loc[:,'obgnme'] = vcount.loc[:,'obgnme'].str.replace('_non_','_non_na_').str.replace('_rmp_','_rmp_na_')
vcount = vcount.obgnme.str.split('_',expand = True).join(vcount)
vcount.columns = ['group', 'deriv', 'depth','zone', 'obgnme','n']
vcount.groupby(['zone',  'deriv']).sum().loc[:,['n']].unstack().droplevel(0,1).plot.barh(title = 'Contribution to Phi Per Group',legend='reverse')

In [ ]:
vcount.groupby(['group','deriv']).sum().loc[:,['n']].unstack().droplevel(0,1).plot.barh()

In [ ]:
pst.observation_data = helpers.set_obs_to_zero_for_manually_selected_obs(pst)

`So to restart, you need to supply the <case>.0.par file to ies_par_en but then you need to pass the obs+noise ensemble from the previous run to ies_obs_en and pass the simulated observation ensemble (the <case>.0.obs. file) to the ies_restart_obs_en argument.  If you do this, ies should be able to work out the alignment and automatically adjust for failed runs.  I know those argument names are confusing - i want to change them but I don't want to break backward compatibility...`

# change these:

<span style="color:blue"> {case}.0.par <-> ies_par_ensemble  
 obs+noise ensemble from the previous run to ies_obs_en  
  (the {case}.0.obs. file) to the ies_restart_obs_en  </span>.


 

In [ ]:
def run_major_pst(obj):
    obj.enforce_bounds()
    del obj.pestpp_options['ies_num_reals']
    del obj.pestpp_options['ies_subset_size'] 
    del obj.pestpp_options['ies_multimodal_alpha']   
    del obj.pestpp_options["ies_bad_phi_sigma"]

    # obj.pestpp_options['ies_num_reals'] = 209 # number of surviving reals
    # obj.pestpp_options["ies_num_reals"] = ies_num_reals  # starting with a real small ensemble!
    obj.pestpp_options['ies_parameter_ensemble'] = f"{run_name.lower()}.0.par.csv"
    obj.pestpp_options['ies_observation_ensemble'] = f"{run_name.lower()}.obs+noise.csv"
    obj.pestpp_options['ies_restart_obs_en'] = f"{run_name.lower()}.0.obs.csv"

    obj.pestpp_options["svd_pack"] = "redsvd"
    obj.pestpp_options["panther_agent_restart_on_error"] = True
    obj.pestpp_options['overdue_giveup_fac'] =  6
    obj.pestpp_options['overdue_giveup_minutes'] =480

    obj.pestpp_options['ies_drop_conflicts'] = True
    obj.control_data.noptmax = 3
    obj.control_data.phiredstp = 1e-20  # just to make sure this doesn't stop the run which has happened
    obj.model_command =[ 'python forward_run.py']

    obj.pestpp_options["ies_bad_phi_sigma"] = 1.5 #middle ground value

run_major_pst(pst)

# remove all old pest files from new folder

In [ ]:
import os
import fnmatch

def remove_files_with_prefix(directory, prefix):
    # Convert prefix to lowercase for case-insensitive comparison
    prefix = prefix.lower()
    
    for filename in os.listdir(directory):
        # Check if the file starts with the prefix (case-insensitive)
        if filename.lower().startswith(prefix):
            file_path = os.path.join(directory, filename)
            if os.path.isfile(file_path):  # Ensure it's a file
                os.remove(file_path)
                print(f"Removed: {file_path}")

# Example usage
prefix_to_remove = 'ipa'          # Replace with your prefix
remove_files_with_prefix(new_run, prefix_to_remove)
        

In [ ]:
# make all non-zero weighted groups have a contribution of 100.0
pst.adjust_weights(obsgrp_dict=og_weights)

# write the pest files

In [ ]:
pst.write(os.path.join(new_run, f'{new_run}.pst'),version=2)

# copy the par and obs files

In [ ]:
obs_file = r"C:\GSP\sv\model\pest_inputs\PEST_PP\pestaws_IPA\ipa.0.obs.csv"
par_file = r"C:\GSP\sv\model\pest_inputs\PEST_PP\pestaws_IPA\ipa.0.par.csv"
noise = r"C:\GSP\sv\model\pest_inputs\PEST_PP\pestaws_IPA\ipa.obs+noise.csv"

shutil.copyfile(par_file, os.path.join(new_run, os.path.basename(par_file)))
shutil.copyfile(obs_file, os.path.join(new_run, os.path.basename(obs_file)))
shutil.copyfile(noise, os.path.join(new_run, os.path.basename(noise)))

In [ ]:
os.listdir(new_run)

In [ ]:
asdf

In [ ]:
obs_file = r"C:\GSP\sv\model\pest_inputs\PEST_PP\pestaws_IPA\ipa.0.obs.csv"
par_file = r"C:\GSP\sv\model\pest_inputs\PEST_PP\pestaws_IPA\ipa.0.par.csv"

o = pd.read_csv(obs_file,nrows = 10)

In [ ]:
p = pd.read_csv(par_file,nrows = 10)

In [ ]:
p

In [ ]:
o

In [ ]:
obs = pst.observation_data.copy()

In [ ]:
obs = pst.observation_data.copy()

gwle = obs.obsnme.str.contains('gwle') | obs.obsnme.str.contains('maj') | obs.obsnme.str.contains('ddown')
hobs = obs.obsnme.str.contains('hds_')
ren = lambda x: x.split("_date:")[0].split(":")[-1] if 'hds' in x or 'gwle' in x else ''
station = obs.obsnme.apply(ren)

date = pd.to_datetime(obs.date)

obs.loc[gwle & date.dt.year<2010]

In [ ]:
date.dt.year.notnull().sum()

In [ ]:
(date.dt.year>1990).sum()

In [ ]:
(date.dt.year>2000).sum()

In [ ]:
(date.dt.year>=2010).sum()

In [ ]:
obs.loc[(date.dt.year>=2010 & date.isnull())]

In [ ]:
obs.loc[gwle & date.dt.year<2010]